In [148]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.io import *
from fastai.conv_learner import *

from fastai.column_data import *

## Setup

We're going to download the collected works of Nietzsche to use as our data for this class.

In [149]:
PATH='data/nietzsche/'

In [150]:
get_data("https://s3.amazonaws.com/text-datasets/nietzsche.txt", f'{PATH}nietzsche.txt')
text = open(f'{PATH}nietzsche.txt').read()
print('corpus length:', len(text))

corpus length: 600893


In [151]:
text[:400]

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not ground\nfor suspecting that all philosophers, in so far as they have been\ndogmatists, have failed to understand women--that the terrible\nseriousness and clumsy importunity with which they have usually paid\ntheir addresses to Truth, have been unskilled and unseemly methods for\nwinning a woman? Certainly she has never allowed herself '

In [152]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 85


Sometimes it's useful to have a zero value in the dataset, e.g. for padding

In [153]:
chars.insert(0, "\0")

''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxy'

Map from chars to indices and back again

In [154]:
char_indices = {c: i for i, c in enumerate(chars)}
indices_char = {i: c for i, c in enumerate(chars)}

*idx* will be the data we use from now own - it simply converts all the characters to their index (based on the mapping above)

In [155]:
idx = [char_indices[c] for c in text]

idx[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

In [156]:
''.join(indices_char[i] for i in idx[:70])

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not gro'

## Three char model

### Create inputs

Create a list of every 4th character, starting at the 0th, 1st, 2nd, then 3rd characters

In [ ]:
cs=3
c1_dat = [idx[i]   for i in range(0, len(idx)-cs, cs)]
c2_dat = [idx[i+1] for i in range(0, len(idx)-cs, cs)]
c3_dat = [idx[i+2] for i in range(0, len(idx)-cs, cs)]
c4_dat = [idx[i+3] for i in range(0, len(idx)-cs, cs)]

Our inputs

In [ ]:
x1 = np.stack(c1_dat)
x2 = np.stack(c2_dat)
x3 = np.stack(c3_dat)

Our output

In [157]:
y = np.stack(c4_dat)

NameError: name 'c4_dat' is not defined

The first 4 inputs and outputs

In [ ]:
x1[:4], x2[:4], x3[:4]

In [ ]:
y[:4]

In [ ]:
x1.shape, y.shape

### Create and train model

Pick a size for our hidden state

In [24]:
n_hidden = 256

The number of latent factors to create (i.e. the size of the embedding matrix)

In [23]:
n_fac = 42

In [ ]:
class Char3Model(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)

        # The 'green arrow' from our diagram - the layer operation from input to hidden
        self.l_in = nn.Linear(n_fac, n_hidden)

        # The 'orange arrow' from our diagram - the layer operation from hidden to hidden
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        
        # The 'blue arrow' from our diagram - the layer operation from hidden to output
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, c1, c2, c3):
        in1 = F.relu(self.l_in(self.e(c1)))
        in2 = F.relu(self.l_in(self.e(c2)))
        in3 = F.relu(self.l_in(self.e(c3)))
        
        h = V(torch.zeros(in1.size()).cuda())
        h = F.tanh(self.l_hidden(h+in1))
        h = F.tanh(self.l_hidden(h+in2))
        h = F.tanh(self.l_hidden(h+in3))
        
        return F.log_softmax(self.l_out(h))

In [ ]:
md = ColumnarModelData.from_arrays('.', [-1], np.stack([x1,x2,x3], axis=1), y, bs=512)

In [ ]:
m = Char3Model(vocab_size, n_fac).cuda()

In [ ]:
it = iter(md.trn_dl)
*xs,yt = next(it)
t = m(*V(xs))

In [ ]:
opt = optim.Adam(m.parameters(), 1e-2)

In [ ]:
fit(m, md, 1, opt, F.nll_loss)

In [ ]:
set_lrs(opt, 0.001)

In [ ]:
fit(m, md, 1, opt, F.nll_loss)

### Test model

In [ ]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [ ]:
get_next('y. ')

In [ ]:
get_next('ppl')

In [ ]:
get_next(' th')

In [ ]:
get_next('and')

## Our first RNN!

### Create inputs

This is the size of our unrolled RNN.

In [10]:
cs=8

For each of 0 through 7, create a list of every 8th character with that starting point. These will be the 8 inputs to out model.

In [11]:
c_in_dat = [[idx[i+j] for i in range(cs)] for j in range(len(idx)-cs)]

Then create a list of the next character in each of these series. This will be the labels for our model.

In [12]:
c_out_dat = [idx[j+cs] for j in range(len(idx)-cs)]

In [25]:
xs = np.stack(c_in_dat, axis=0)

In [26]:
xs.shape

(600885, 8)

In [27]:
y = np.stack(c_out_dat)

So each column below is one series of 8 characters from the text.

In [28]:
xs[:cs,:cs]

array([[40, 42, 29, 30, 25, 27, 29,  1],
       [42, 29, 30, 25, 27, 29,  1,  1],
       [29, 30, 25, 27, 29,  1,  1,  1],
       [30, 25, 27, 29,  1,  1,  1, 43],
       [25, 27, 29,  1,  1,  1, 43, 45],
       [27, 29,  1,  1,  1, 43, 45, 40],
       [29,  1,  1,  1, 43, 45, 40, 40],
       [ 1,  1,  1, 43, 45, 40, 40, 39]])

...and this is the next character after each sequence.

In [29]:
y[:cs]

array([ 1,  1, 43, 45, 40, 40, 39, 43])

### Create and train model

In [30]:
val_idx = get_cv_idxs(len(idx)-cs-1)

In [31]:
md = ColumnarModelData.from_arrays('.', val_idx, xs, y, bs=512)

In [108]:
class CharLoopModel(nn.Module):
    # This is an RNN!
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.l_in = nn.Linear(n_fac, n_hidden)
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        print('CS:', len(cs))
        print('CS:', cs[0].shape)
        print('CS:', cs[1].shape)
        bs = cs[0].size(0)
        print(bs)
        h = V(torch.zeros(bs, n_hidden).cuda())
        for c in cs:
            inp = F.relu(self.l_in(self.e(c)))
            h = F.tanh(self.l_hidden(h+inp))
        print(h.shape)
        print(self.l_out(h).shape)
        return F.log_softmax(self.l_out(h), dim=-1)

In [109]:
m = CharLoopModel(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-2)
stepper = Stepper(m, opt, F.nll_loss)

In [110]:
for (*x,y) in iter(md.trn_dl):
    m(*V(x))
#     stepper.step(V(x),V(y))
    break

CS: 8
CS: torch.Size([512])
CS: torch.Size([512])
512
torch.Size([512, 256])
torch.Size([512, 85])


In [99]:
fit(m, md, 1, opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

  0%|          | 0/939 [00:00<?, ?it/s]512                 
torch.Size([512, 256])
torch.Size([512, 85])
  0%|          | 1/939 [00:00<04:41,  3.33it/s, loss=4.5]512
torch.Size([512, 256])
torch.Size([512, 85])
  0%|          | 1/939 [00:00<05:28,  2.86it/s, loss=4.09]512
torch.Size([512, 256])
torch.Size([512, 85])
  0%|          | 3/939 [00:00<02:05,  7.47it/s, loss=3.77]512
torch.Size([512, 256])
torch.Size([512, 85])
  0%|          | 3/939 [00:00<02:23,  6.53it/s, loss=3.72]512
torch.Size([512, 256])
torch.Size([512, 85])
  0%|          | 3/939 [00:00<02:35,  6.01it/s, loss=3.57]512
torch.Size([512, 256])
torch.Size([512, 85])
  1%|          | 6/939 [00:00<01:23, 11.24it/s, loss=3.44]512
torch.Size([512, 256])
torch.Size([512, 85])
  1%|          | 6/939 [00:00<01:28, 10.53it/s, loss=3.36]512
torch.Size([512, 256])
torch.Size([512, 85])
  1%|          | 6/939 [00:00<01:33,  9.94it/s, loss=3.28]512
torch.Size([512, 256])
torch.Size([512, 85])
  1%|          | 9/939 [00:00<01:06, 13.

KeyboardInterrupt: 

  3%|▎         | 27/939 [00:13<07:48,  1.95it/s, loss=2.79]

In [ ]:
set_lrs(opt, 0.001)

In [ ]:
fit(m, md, 1, opt, F.nll_loss)

In [ ]:
class CharLoopConcatModel(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.l_in = nn.Linear(n_fac+n_hidden, n_hidden)
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(bs, n_hidden).cuda())
        for c in cs:
            inp = torch.cat((h, self.e(c)), 1)
            inp = F.relu(self.l_in(inp))
            h = F.tanh(self.l_hidden(inp))
        
        return F.log_softmax(self.l_out(h), dim=-1)

In [ ]:
m = CharLoopConcatModel(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [ ]:
it = iter(md.trn_dl)
*xs,yt = next(it)
t = m(*V(xs))

In [ ]:
fit(m, md, 1, opt, F.nll_loss)

In [ ]:
set_lrs(opt, 1e-4)

In [ ]:
fit(m, md, 1, opt, F.nll_loss)

### Test model

In [ ]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [ ]:
get_next('for thos')

In [ ]:
get_next('part of ')

In [ ]:
get_next('queens a')

## RNN with pytorch

In [121]:
class CharRnn(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden, batch_first=True)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(1, bs, n_hidden))
        inp = self.e(torch.stack(cs))
        print('inp:', inp.shape)
        outp,h = self.rnn(inp, h)
        print('outp:', outp.shape)
        return F.log_softmax(self.l_out(outp[-1]), dim=-1)

In [ ]:
m = CharRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [ ]:
it = iter(md.trn_dl)
*xs,yt = next(it)

In [ ]:
t = m.e(V(torch.stack(xs)))
t.size()

In [ ]:
ht = V(torch.zeros(1, 512,n_hidden))
outp, hn = m.rnn(t, ht)
outp.size(), hn.size()

In [ ]:
t = m(*V(xs)); t.size()

In [ ]:
fit(m, md, 4, opt, F.nll_loss)

In [ ]:
set_lrs(opt, 1e-4)

In [ ]:
fit(m, md, 2, opt, F.nll_loss)

### Test model

In [ ]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [ ]:
get_next('for thos')

In [ ]:
def get_next_n(inp, n):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [ ]:
get_next_n('for thos', 40)

## Multi-output model

### Setup

Let's take non-overlapping sets of characters this time

In [230]:
c_in_dat = [[idx[i+j] for i in range(cs)] for j in range(0, len(idx)-cs-1, cs)]

Then create the exact same thing, offset by 1, as our labels

In [231]:
c_out_dat = [[idx[i+j] for i in range(cs)] for j in range(1, len(idx)-cs, cs)]

In [232]:
xs = np.stack(c_in_dat)
xs.shape

(75111, 8)

In [233]:
ys = np.stack(c_out_dat)
ys.shape

(75111, 8)

In [234]:
xs[:cs,:cs]

array([[40, 42, 29, 30, 25, 27, 29,  1],
       [ 1,  1, 43, 45, 40, 40, 39, 43],
       [33, 38, 31,  2, 73, 61, 54, 73],
       [ 2, 44, 71, 74, 73, 61,  2, 62],
       [72,  2, 54,  2, 76, 68, 66, 54],
       [67,  9,  9, 76, 61, 54, 73,  2],
       [73, 61, 58, 67, 24,  2, 33, 72],
       [ 2, 73, 61, 58, 71, 58,  2, 67]])

In [235]:
ys[:cs,:cs]

array([[42, 29, 30, 25, 27, 29,  1,  1],
       [ 1, 43, 45, 40, 40, 39, 43, 33],
       [38, 31,  2, 73, 61, 54, 73,  2],
       [44, 71, 74, 73, 61,  2, 62, 72],
       [ 2, 54,  2, 76, 68, 66, 54, 67],
       [ 9,  9, 76, 61, 54, 73,  2, 73],
       [61, 58, 67, 24,  2, 33, 72,  2],
       [73, 61, 58, 71, 58,  2, 67, 68]])

### Create and train model

In [132]:
val_idx = get_cv_idxs(len(xs)-cs-1)

In [133]:
md = ColumnarModelData.from_arrays('.', val_idx, xs, ys, bs=512)

In [ ]:
class CharSeqRnn(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(1, bs, n_hidden))
        inp = self.e(torch.stack(cs))
        outp,h = self.rnn(inp, h)
        return F.log_softmax(self.l_out(outp), dim=-1)

In [ ]:
m = CharSeqRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [ ]:
it = iter(md.trn_dl)
*xst,yt = next(it)

In [ ]:
def nll_loss_seq(inp, targ):
    sl,bs,nh = inp.size()
    targ = targ.transpose(0,1).contiguous().view(-1)
    return F.nll_loss(inp.view(-1,nh), targ)

In [ ]:
fit(m, md, 4, opt, nll_loss_seq)

In [ ]:
set_lrs(opt, 1e-4)

In [ ]:
fit(m, md, 1, opt, nll_loss_seq)

### Identity init!

In [ ]:
m = CharSeqRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-2)

In [ ]:
m.rnn.weight_hh_l0.data.copy_(torch.eye(n_hidden))

In [ ]:
fit(m, md, 4, opt, nll_loss_seq)

In [ ]:
set_lrs(opt, 1e-3)

In [ ]:
fit(m, md, 4, opt, nll_loss_seq)

## Stateful model

### Setup

In [158]:
from torchtext import vocab, data

from fastai.nlp import *
from fastai.lm_rnn import *

PATH='data/nietzsche/'

TRN_PATH = 'trn/'
VAL_PATH = 'val/'
TRN_PATH = ''
VAL_PATH = ''
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

%ls {PATH}

nietzsche.txt


In [159]:
%ls {PATH}trn

ls: cannot access 'data/nietzsche/trn': No such file or directory


In [160]:
TEXT = data.Field(lower=True, tokenize=list)
bs=64; bptt=8; n_fac=42; n_hidden=256

FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=3)

len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(1153, 55, 1, 590960)

### RNN

In [ ]:
class CharSeqStatefulRnn(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        self.vocab_size = vocab_size
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h.size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs): self.h = V(torch.zeros(1, bs, n_hidden))

In [ ]:
m = CharSeqStatefulRnn(md.nt, n_fac, 512).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [ ]:
fit(m, md, 4, opt, F.nll_loss)

In [ ]:
set_lrs(opt, 1e-4)

fit(m, md, 4, opt, F.nll_loss)

### RNN loop

In [ ]:
# From the pytorch source

def RNNCell(input, hidden, w_ih, w_hh, b_ih, b_hh):
    return F.tanh(F.linear(input, w_ih, b_ih) + F.linear(hidden, w_hh, b_hh))

In [ ]:
class CharSeqStatefulRnn2(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        super().__init__()
        self.vocab_size = vocab_size
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNNCell(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h.size(1) != bs: self.init_hidden(bs)
        outp = []
        o = self.h
        for c in cs: 
            o = self.rnn(self.e(c), o)
            outp.append(o)
        outp = self.l_out(torch.stack(outp))
        self.h = repackage_var(o)
        return F.log_softmax(outp, dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs): self.h = V(torch.zeros(1, bs, n_hidden))

In [ ]:
m = CharSeqStatefulRnn2(md.nt, n_fac, 512).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [ ]:
fit(m, md, 4, opt, F.nll_loss)

### GRU

In [ ]:
class CharSeqStatefulGRU(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        super().__init__()
        self.vocab_size = vocab_size
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.GRU(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h.size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs): self.h = V(torch.zeros(1, bs, n_hidden))

In [ ]:
# From the pytorch source code - for reference

def GRUCell(input, hidden, w_ih, w_hh, b_ih, b_hh):
    gi = F.linear(input, w_ih, b_ih)
    gh = F.linear(hidden, w_hh, b_hh)
    i_r, i_i, i_n = gi.chunk(3, 1)
    h_r, h_i, h_n = gh.chunk(3, 1)

    resetgate = F.sigmoid(i_r + h_r)
    inputgate = F.sigmoid(i_i + h_i)
    newgate = F.tanh(i_n + resetgate * h_n)
    return newgate + inputgate * (hidden - newgate)

In [ ]:
m = CharSeqStatefulGRU(md.nt, n_fac, 512).cuda()

opt = optim.Adam(m.parameters(), 1e-3)

In [ ]:
fit(m, md, 6, opt, F.nll_loss)

In [ ]:
set_lrs(opt, 1e-4)

In [ ]:
fit(m, md, 3, opt, F.nll_loss)

### Putting it all together: LSTM

In [161]:
from fastai import sgdr

n_hidden=512

In [178]:
class CharSeqStatefulLSTM(nn.Module):
    def __init__(self, vocab_size, n_fac, bs, nl):
        super().__init__()
        self.vocab_size,self.nl = vocab_size,nl
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.LSTM(n_fac, n_hidden, nl, dropout=0.5)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        print(cs.shape)
        bs = cs[0].size(0)
        print('BS:', bs)
        if self.h[0].size(1) != bs: self.init_hidden(bs)
        e = self.e(cs)
        print('E', e.shape)
        outp,h = self.rnn(e, self.h)
        print('Outp:', outp.shape)
        print('h:', h[0].shape)
        self.h = repackage_var(h)
        print('lout:', self.l_out(outp).shape)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs):
        print('BS:', bs)
        self.h = (V(torch.zeros(self.nl, bs, n_hidden)),
                  V(torch.zeros(self.nl, bs, n_hidden)))

In [222]:
TEXT = data.Field(lower=True, tokenize=list)
bs=12; bptt=6; n_fac=42; n_hidden=256

FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=3)

len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(8206, 55, 1, 590960)

In [223]:
n_fac

42

In [224]:
md.nt

55

In [225]:
m = CharSeqStatefulLSTM(md.nt, n_fac, 512, 1).cuda()
lo = LayerOptimizer(optim.Adam, m, 1e-2, 1e-5)
stepper = Stepper(m, lo, F.nll_loss)

BS: 512


In [226]:
it = iter(md.trn_dl)

In [227]:
for (*x,y) in it:
    print('xshape:', x[0].shape)
    out = m(*V(x))
    print('yshape:', y.shape)
    print('outshape:', out.shape)
#     stepper.step(V(x),V(y))
    break

xshape: torch.Size([5, 12])
torch.Size([5, 12])
BS: 12
BS: 12
E torch.Size([5, 12, 42])
Outp: torch.Size([5, 12, 256])
h: torch.Size([1, 12, 256])
lout: torch.Size([5, 12, 55])
yshape: torch.Size([60])
outshape: torch.Size([60, 55])


In [194]:
5 * 64

320

In [228]:
x

[Variable containing:
    18    20     6    17     5     5     8     9     6     2     2    17
    10     2    10     2    17    17    14    17    27     5     5    18
     3     7     4     7     3    18    12     6     5     8     4     4
    16     8     3    16     9    15    15     8     8     2    21    20
     6     2    13     2     2    12     9    22    19     4     3     2
 [torch.cuda.LongTensor of size 5x12 (GPU 0)]]

In [236]:
y.view(5, 12)

Variable containing:
   10     2    10     2    17    17    14    17    27     5     5    18
    3     7     4     7     3    18    12     6     5     8     4     4
   16     8     3    16     9    15    15     8     8     2    21    20
    6     2    13     2     2    12     9    22    19     4     3     2
   14     4    22    15    21     9     5     2    22    11    10    18
[torch.cuda.LongTensor of size 5x12 (GPU 0)]

In [193]:
out

Variable containing:
-3.9731 -3.9724 -3.9818  ...  -3.9970 -4.0048 -4.0160
-3.9702 -3.9809 -3.9715  ...  -4.0018 -4.0112 -4.0287
-3.9761 -3.9853 -3.9725  ...  -3.9858 -3.9955 -4.0228
          ...             ⋱             ...          
-3.9776 -3.9585 -3.9856  ...  -3.9964 -4.0037 -4.0226
-3.9748 -3.9770 -3.9810  ...  -4.0050 -3.9968 -4.0187
-3.9814 -3.9798 -3.9867  ...  -4.0025 -3.9956 -4.0257
[torch.cuda.FloatTensor of size 320x55 (GPU 0)]

In [192]:
y

Variable containing:
 10
 10
 20
  4
  3
 10
  2
  2
  2
  7
  9
 44
 17
  4
 27
  5
  3
 21
  6
  4
 16
  4
  5
 10
 15
  6
  7
  2
  2
 11
 22
  8
 23
  2
  5
 26
  2
  5
 20
 22
  8
 51
  7
  5
  7
 12
  2
 22
 26
 10
  2
 21
  4
  2
  8
  7
  3
 12
 37
  2
 13
  3
  3
  3
  3
 20
  2
 23
  2
  2
  3
  7
  6
  2
  2
 26
  7
 11
  3
  4
 10
  8
  5
  3
  2
  2
  4
  3
 10
 21
 10
  6
  4
  2
  2
  6
  6
  6
 24
  2
 16
  4
  2
  2
  2
  2
 12
  7
  8
 12
  4
  2
  2
 20
 21
  5
  2
 10
 19
  7
  2
 20
 26
 21
  7
  2
 10
  9
 16
  2
  3
  3
 27
  3
 24
  8
  9
  9
  5
  2
 13
  5
  3
 20
 20
  5
  8
 10
  4
  9
  2
  6
 18
  6
  4
  8
  7
 18
  4
  8
 23
 10
  5
  3
 15
  9
 23
  6
 14
 13
 13
  8
  4
 13
  7
 14
  4
 22
  5
  4
  5
  3
  2
 27
  6
  5
  2
  3
  2
 10
  8
  2
  6
 16
 24
  2
  8
  8
  3
  3
  4
 15
  8
 21
  3
  8
 18
  2
  2
 13
  4
 18
 11
  3
 21
 13
  7
 27
 11
 13
  2
  7
 11
 14
 12
 23
 13
 24
  4
  3
  7
  8
  5
  3
  8
  2
 10
  3
  2
 15
 11
  2
  4
 11
  9

In [177]:
m = CharSeqStatefulLSTM(md.nt, n_fac, 512, 2).cuda()
lo = LayerOptimizer(optim.Adam, m, 1e-2, 1e-5)

BS: 512


In [ ]:
os.makedirs(f'{PATH}models', exist_ok=True)

In [ ]:
fit(m, md, 2, lo.opt, F.nll_loss)

In [ ]:
on_end = lambda sched, cycle: save_model(m, f'{PATH}models/cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
fit(m, md, 2**4-1, lo.opt, F.nll_loss, callbacks=cb)

In [ ]:
on_end = lambda sched, cycle: save_model(m, f'{PATH}models/cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
fit(m, md, 2**6-1, lo.opt, F.nll_loss, callbacks=cb)

### Test

In [ ]:
def get_next(inp):
    idxs = TEXT.numericalize(inp)
    p = m(VV(idxs.transpose(0,1)))
    r = torch.multinomial(p[-1].exp(), 1)
    return TEXT.vocab.itos[to_np(r)[0]]

In [ ]:
get_next('for thos')

In [ ]:
def get_next_n(inp, n):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [ ]:
print(get_next_n('for thos', 400))